# Chr1-22, s40.22, GTEx Adipose (subcutaneous) fusion weights

This uses genotype chr1-22, filtered 40000 to 22k samples.

## Part1: simulation and mr.ash

* set up

In [1]:
wkdir = "/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20201001/"

In [2]:
codedir = "/project2/mstephens/causalTWAS/causal-TWAS/code/"

In [3]:
geno = "/project2/mstephens/causalTWAS/ukbiobank/ukb_chr1to22_s40.22_fn.txt"

In [4]:
weight = "/project2/mstephens/causalTWAS/fusion_weights/Adipose_Subcutaneous"

In [5]:
maintag = '20201001-2'

In [6]:
subtags =  [str(i) for i in range(1,6)]

In [7]:
simu_expr_name = "simu_s40.22_GTEXadipose"

In [8]:
simu_expr = "{simu_expr_name}-cis-expr.txt".format(simu_expr_name = simu_expr_name)

* prepare param 

In [9]:
cd {wkdir}

/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20201001


In [10]:
template = "{wkdir}/param-{maintag}.R".format(wkdir = wkdir, maintag = maintag)

In [11]:
a = open(template, 'r').readlines()

In [12]:
for subtag in subtags:
    with open("param-{maintag}-{subtag}.R".format(maintag = maintag, subtag = subtag), 'w') as newpar:
        a[5] = 'SED <- {sed}\n'.format(sed = str(950 + int(subtag)))
        newpar.writelines(a)

* commands

In [13]:
g2dir = "cd {wkdir}"

In [14]:
impu = "Rscript {codedir}/run_impute_expr.R {geno} {weight} {simu_expr_name}"

In [15]:
simu = "Rscript {codedir}/run_simulate_pheno.R {geno} {simu_expr} param-{tag}.R simu_{tag}"

In [16]:
simu_pheno = "{wkdir}/simu_{tag}-pheno.Rd"

In [17]:
exprgwas = "Rscript {codedir}/run_gwas_expr.R {simu_expr} {simu_pheno} {tag}"

In [18]:
snpgwas = "Rscript {codedir}/run_gwas_snp.R {geno} {simu_pheno} {tag}"

In [19]:
ld = "Rscript {codedir}/run_ld.R {geno} {simu_expr} {simu_pheno} {tag}"

* write master file

In [20]:
tags = [maintag + '-' + subtag for subtag in subtags]

In [21]:
masterimpufile = 'master_impu.sh'

In [22]:
cd {wkdir}

/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20201001


In [23]:
with open(masterimpufile, 'w') as outf:
        outf.write(';'.join([
              'source activate twas', 
              g2dir,
              impu
        ]).format(
              simu_expr_name = simu_expr_name,
              simu_expr = simu_expr,
              simu_pheno = simu_pheno,
              wkdir = wkdir,
              codedir = codedir,
              geno = geno,
              weight = weight
        ).format(
              wkdir = wkdir,
        ) + '\n')

In [24]:
!python /home/simingz/cancer_somatic/cancer_somatic/annotation/pMake_batchjobs_RCC.py {masterimpufile} --mem 20 --ntasks 1 --partition mstephens

planning to run 1 jobs ...
Type: sbatch submit_parallel_master_impu.sh.sh


In [25]:
masterfile = 'master_run2.sh'

In [26]:
with open(masterfile, 'w') as outf:
    for tag in tags:
        outf.write(';'.join([
              'source activate twas', 
              g2dir,
              simu,
              exprgwas,
              snpgwas,
              #ld
        ]).format(
              simu_expr_name = simu_expr_name,
              simu_expr = simu_expr,
              simu_pheno = simu_pheno,
              wkdir = wkdir,
              codedir = codedir,
              geno = geno,
              weight = weight,
              tag = tag
        ).format(
              wkdir = wkdir,
              tag = tag
        ) + '\n')

In [27]:
!python /home/simingz/cancer_somatic/cancer_somatic/annotation/pMake_batchjobs_RCC.py {masterfile} --mem 20 --ntasks 1 --partition mstephens

planning to run 5 jobs ...
Type: sbatch submit_parallel_master_run2.sh.sh
